# Setup

In [62]:
%load_ext kedro.ipython
import pandas as pd
import numpy as np

from thefuzz import fuzz

The kedro.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.ipython


In [63]:
%reload_kedro
df = catalog.load("prim_mibolsillo")
df

[04/02/24 20:54:54] INFO     Loading data from prim_mibolsillo (ParquetDataset)...              ]8;id=923224;file:///usr/local/Caskroom/miniforge/base/envs/kedro-environment/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=384412;file:///usr/local/Caskroom/miniforge/base/envs/kedro-environment/lib/python3.10/site-packages/kedro/io/data_catalog.py#483\483]8;;\

,id,branch_number,city,state,age,gender,total_credit_card_limit,current_available_limit,date,amount,category_expense,purchase_city,purchase_country,geobr_purchase_city,geobr_purchase_city_conf,geobr_purchase_state
0,"4,53E+11",201405,CAMPO LIMPO PAULISTA,SP,37,F,4700,5605,2019-12-04,31.0,SERVI‚O,SAO PAULO,BR,São Paulo,0.94,SP
1,"4,53E+11",201405,CAMPO LIMPO PAULISTA,SP,37,F,4700,5343,2019-11-09,15001.0,FARMACIAS,SANTOS,BR,Santos,1.00,SP
2,"4,53E+11",201405,CAMPO LIMPO PAULISTA,SP,37,F,4700,2829,2019-05-06,50.0,SERVI‚O,SAO PAULO,BR,São Paulo,0.94,SP
3,"4,53E+11",201405,CAMPO LIMPO PAULISTA,SP,37,F,4700,2547,2019-06-01,544.0,M.O.T.O.,OSASCO,BR,Osasco,1.00,SP
4,"4,53E+11",201405,CAMPO LIMPO PAULISTA,SP,37,F,4700,2515,2019-06-01,3279.0,M.O.T.O.,OSASCO,BR,Osasco,1.00,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4950,"9,19E+11",201910,SAO PAULO,SP,37,M,8500,5305,2020-01-23,255.0,RESTAURANTE,SAO PAULO,BR,São Paulo,0.94,SP
4951,"9,19E+11",201910,SAO PAULO,SP,37,M,8500,5266,2020-01-23,395.0,SUPERMERCADOS,SAO PAULO,BR,São Paulo,0.94,SP
4952,"9,19E+11",201910,SAO PAULO,SP,37,M,8500,5445,2020-02-19,7556.0,RESTAURANTE,SAO PAULO,BR,São Paulo,0.94,SP
4953,"9,19E+11",201910,SAO PAULO,SP,37,M,8500,5002,2020-03-30,59.0,SERVI‚O,SAO PAULO,BR,São Paulo,0.94,SP


# Feature Engineering

In [64]:
model_data = df[['total_credit_card_limit', 'current_available_limit', 'amount']] # take only numerical columns
model_data.head()

,total_credit_card_limit,current_available_limit,amount
0,4700,5605,31.0
1,4700,5343,15001.0
2,4700,2829,50.0
3,4700,2547,544.0
4,4700,2515,3279.0


In [65]:
model_data['total_minus_current'] = model_data['total_credit_card_limit'] - model_data['current_available_limit']
model_data['current_available_limit_ratio'] = model_data['current_available_limit'] / model_data['total_credit_card_limit']
model_data['amount_ratio_from_total'] = model_data['amount'] / model_data['total_credit_card_limit']
model_data['amount_ratio_from_current'] = model_data['amount'] / model_data['current_available_limit']
model_data.head()


[04/02/24 20:54:55] WARNING  /var/folders/qn/pq9pzsn51vx7lzwtkc1r1hxr0000gn/T/ipykernel_2392/307283 ]8;id=514916;file:///usr/local/Caskroom/miniforge/base/envs/kedro-environment/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=821399;file:///usr/local/Caskroom/miniforge/base/envs/kedro-environment/lib/python3.10/warnings.py#109\109]8;;\
                             3996.py:1: SettingWithCopyWarning:                                                    
                             A value is trying to be set on a copy of a slice from a DataFrame.                    
                             Try using .loc[row_indexer,col_indexer] = value instead                               
                                                                                                                   
                             See the caveats in the documentation:                                                 
                             https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#                
                             returning-a-view-versus-a-copy                                                        
                               model_data['total_minus_current'] =                                                 
                             model_data['total_credit_card_limit'] -                                               
                             model_data['current_available_limit']                                                 
                                                                                                                   

                    WARNING  /var/folders/qn/pq9pzsn51vx7lzwtkc1r1hxr0000gn/T/ipykernel_2392/307283 ]8;id=829914;file:///usr/local/Caskroom/miniforge/base/envs/kedro-environment/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=695514;file:///usr/local/Caskroom/miniforge/base/envs/kedro-environment/lib/python3.10/warnings.py#109\109]8;;\
                             3996.py:2: SettingWithCopyWarning:                                                    
                             A value is trying to be set on a copy of a slice from a DataFrame.                    
                             Try using .loc[row_indexer,col_indexer] = value instead                               
                                                                                                                   
                             See the caveats in the documentation:                                                 
                             https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#                
                             returning-a-view-versus-a-copy                                                        
                               model_data['current_available_limit_ratio'] =                                       
                             model_data['current_available_limit'] /                                               
                             model_data['total_credit_card_limit']                                                 
                                                                                                                   

                    WARNING  /var/folders/qn/pq9pzsn51vx7lzwtkc1r1hxr0000gn/T/ipykernel_2392/307283 ]8;id=472874;file:///usr/local/Caskroom/miniforge/base/envs/kedro-environment/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=67952;file:///usr/local/Caskroom/miniforge/base/envs/kedro-environment/lib/python3.10/warnings.py#109\109]8;;\
                             3996.py:3: SettingWithCopyWarning:                                                    
                             A value is trying to be set on a copy of a slice from a DataFrame.                    
                             Try using .loc[row_indexer,col_indexer] = value instead                               
                                                                                                                   
                             See the caveats in the documentation:                                                 
                             https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#                
                             returning-a-view-versus-a-copy                                                        
                               model_data['amount_ratio_from_total'] = model_data['amount'] /                      
                             model_data['total_credit_card_limit']                                                 
                                                                                                                   

                    WARNING  /var/folders/qn/pq9pzsn51vx7lzwtkc1r1hxr0000gn/T/ipykernel_2392/307283 ]8;id=974182;file:///usr/local/Caskroom/miniforge/base/envs/kedro-environment/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=236409;file:///usr/local/Caskroom/miniforge/base/envs/kedro-environment/lib/python3.10/warnings.py#109\109]8;;\
                             3996.py:4: SettingWithCopyWarning:                                                    
                             A value is trying to be set on a copy of a slice from a DataFrame.                    
                             Try using .loc[row_indexer,col_indexer] = value instead                               
                                                                                                                   
                             See the caveats in the documentation:                                                 
                             https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#                
                             returning-a-view-versus-a-copy                                                        
                               model_data['amount_ratio_from_current'] = model_data['amount'] /                    
                             model_data['current_available_limit']                                                 
                                                                                                                   

,total_credit_card_limit,current_available_limit,amount,total_minus_current,current_available_limit_ratio,amount_ratio_from_total,amount_ratio_from_current
0,4700,5605,31.0,-905,1.192553,0.006596,0.005531
1,4700,5343,15001.0,-643,1.136809,3.191702,2.807599
2,4700,2829,50.0,1871,0.601915,0.010638,0.017674
3,4700,2547,544.0,2153,0.541915,0.115745,0.213585
4,4700,2515,3279.0,2185,0.535106,0.697660,1.303777


In [66]:
model_data['is_F'] = np.where(df.gender == 'F', 1, 0)
model_data['is_M'] = np.where(df.gender == 'M', 1, 0)
model_data.head()

                    WARNING  /var/folders/qn/pq9pzsn51vx7lzwtkc1r1hxr0000gn/T/ipykernel_2392/218784 ]8;id=714736;file:///usr/local/Caskroom/miniforge/base/envs/kedro-environment/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=840004;file:///usr/local/Caskroom/miniforge/base/envs/kedro-environment/lib/python3.10/warnings.py#109\109]8;;\
                             5388.py:1: SettingWithCopyWarning:                                                    
                             A value is trying to be set on a copy of a slice from a DataFrame.                    
                             Try using .loc[row_indexer,col_indexer] = value instead                               
                                                                                                                   
                             See the caveats in the documentation:                                                 
                             https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#                
                             returning-a-view-versus-a-copy                                                        
                               model_data['is_F'] = np.where(df.gender == 'F', 1, 0)                               
                                                                                                                   

                    WARNING  /var/folders/qn/pq9pzsn51vx7lzwtkc1r1hxr0000gn/T/ipykernel_2392/218784 ]8;id=257836;file:///usr/local/Caskroom/miniforge/base/envs/kedro-environment/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=603296;file:///usr/local/Caskroom/miniforge/base/envs/kedro-environment/lib/python3.10/warnings.py#109\109]8;;\
                             5388.py:2: SettingWithCopyWarning:                                                    
                             A value is trying to be set on a copy of a slice from a DataFrame.                    
                             Try using .loc[row_indexer,col_indexer] = value instead                               
                                                                                                                   
                             See the caveats in the documentation:                                                 
                             https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#                
                             returning-a-view-versus-a-copy                                                        
                               model_data['is_M'] = np.where(df.gender == 'M', 1, 0)                               
                                                                                                                   

,total_credit_card_limit,current_available_limit,amount,total_minus_current,current_available_limit_ratio,amount_ratio_from_total,amount_ratio_from_current,is_F,is_M
0,4700,5605,31.0,-905,1.192553,0.006596,0.005531,1,0
1,4700,5343,15001.0,-643,1.136809,3.191702,2.807599,1,0
2,4700,2829,50.0,1871,0.601915,0.010638,0.017674,1,0
3,4700,2547,544.0,2153,0.541915,0.115745,0.213585,1,0
4,4700,2515,3279.0,2185,0.535106,0.697660,1.303777,1,0


In [67]:
# model_data['is_the_same_city'] = df[['city', 'purchase_city']].apply(lambda city, purchase_city: fuzz.ratio(city, purchase_city))
model_data['is_the_same_city'] = df.apply(lambda row: fuzz.ratio(row.city, row.purchase_city)/100, axis=1)
model_data['is_the_state'] = df.apply(lambda row: fuzz.ratio(row.state, row.geobr_purchase_state)/100, axis=1)
model_data['is_the_state_conf'] = model_data['is_the_state'] * df['geobr_purchase_city_conf']
model_data.head()

[04/02/24 20:54:56] WARNING  /var/folders/qn/pq9pzsn51vx7lzwtkc1r1hxr0000gn/T/ipykernel_2392/126066 ]8;id=56793;file:///usr/local/Caskroom/miniforge/base/envs/kedro-environment/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=227090;file:///usr/local/Caskroom/miniforge/base/envs/kedro-environment/lib/python3.10/warnings.py#109\109]8;;\
                             8859.py:2: SettingWithCopyWarning:                                                    
                             A value is trying to be set on a copy of a slice from a DataFrame.                    
                             Try using .loc[row_indexer,col_indexer] = value instead                               
                                                                                                                   
                             See the caveats in the documentation:                                                 
                             https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#                
                             returning-a-view-versus-a-copy                                                        
                               model_data['is_the_same_city'] = df.apply(lambda row:                               
                             fuzz.ratio(row.city, row.purchase_city)/100, axis=1)                                  
                                                                                                                   

                    WARNING  /var/folders/qn/pq9pzsn51vx7lzwtkc1r1hxr0000gn/T/ipykernel_2392/126066 ]8;id=342829;file:///usr/local/Caskroom/miniforge/base/envs/kedro-environment/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=847152;file:///usr/local/Caskroom/miniforge/base/envs/kedro-environment/lib/python3.10/warnings.py#109\109]8;;\
                             8859.py:3: SettingWithCopyWarning:                                                    
                             A value is trying to be set on a copy of a slice from a DataFrame.                    
                             Try using .loc[row_indexer,col_indexer] = value instead                               
                                                                                                                   
                             See the caveats in the documentation:                                                 
                             https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#                
                             returning-a-view-versus-a-copy                                                        
                               model_data['is_the_state'] = df.apply(lambda row:                                   
                             fuzz.ratio(row.state, row.geobr_purchase_state)/100, axis=1)                          
                                                                                                                   

                    WARNING  /var/folders/qn/pq9pzsn51vx7lzwtkc1r1hxr0000gn/T/ipykernel_2392/126066 ]8;id=442833;file:///usr/local/Caskroom/miniforge/base/envs/kedro-environment/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=943666;file:///usr/local/Caskroom/miniforge/base/envs/kedro-environment/lib/python3.10/warnings.py#109\109]8;;\
                             8859.py:4: SettingWithCopyWarning:                                                    
                             A value is trying to be set on a copy of a slice from a DataFrame.                    
                             Try using .loc[row_indexer,col_indexer] = value instead                               
                                                                                                                   
                             See the caveats in the documentation:                                                 
                             https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#                
                             returning-a-view-versus-a-copy                                                        
                               model_data['is_the_state_conf'] = model_data['is_the_state'] *                      
                             df['geobr_purchase_city_conf']                                                        
                                                                                                                   

,total_credit_card_limit,current_available_limit,amount,total_minus_current,current_available_limit_ratio,amount_ratio_from_total,amount_ratio_from_current,is_F,is_M,is_the_same_city,is_the_state,is_the_state_conf
0,4700,5605,31.0,-905,1.192553,0.006596,0.005531,1,0,0.48,1.0,0.94
1,4700,5343,15001.0,-643,1.136809,3.191702,2.807599,1,0,0.23,1.0,1.00
2,4700,2829,50.0,1871,0.601915,0.010638,0.017674,1,0,0.48,1.0,0.94
3,4700,2547,544.0,2153,0.541915,0.115745,0.213585,1,0,0.23,1.0,1.00
4,4700,2515,3279.0,2185,0.535106,0.697660,1.303777,1,0,0.23,1.0,1.00


# Labels

Due to the significant imbalance in label counts, I opted for a more meaningful grouping approach: grouping by purpose. Here's the breakdown:

- Necessities: SERVI‚O, SUPERMERCADOS, FARMACIAS, POSTO DE GAS, M.O.T.O.
- Discretionary: RESTAURANTE, VAREJO, VESTUARIO, ARTIGOS ELETRO, MOVEIS E DECOR, AUTO PE‚AS, JOALHERIA
- Travel: CIA AEREAS, HOTEIS, AGENCIA DE TUR"

In [68]:
df['category_expense'].value_counts()


category_expense
SERVI‚O           1769
RESTAURANTE        871
VAREJO             599
SUPERMERCADOS      468
VESTUARIO          236
FARMACIAS          222
POSTO DE GAS       199
M.O.T.O.           177
ARTIGOS ELETRO     122
LOJA DE DEPART      80
CIA AEREAS          36
HOTEIS              24
HOSP E CLINICA      18
SEM RAMO            16
MOVEIS E DECOR      15
MAT CONSTRUCAO      15
AUTO PE‚AS          14
AGENCIA DE TUR      12
JOALHERIA            6
ALUG DE CARROS       1
Name: count, dtype: int64

In [71]:
%reload_kedro

[04/02/24 20:56:30] INFO     Resolved project path as:                                              ]8;id=579711;file:///usr/local/Caskroom/miniforge/base/envs/kedro-environment/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=672357;file:///usr/local/Caskroom/miniforge/base/envs/kedro-environment/lib/python3.10/site-packages/kedro/ipython/__init__.py#164\164]8;;\
                             /Users/richarddegaki/repos/xrhd/brazilian-credit-card-spending.                       
                             To set a different path, run '%reload_kedro <project_root>'                           

[04/02/24 20:56:31] INFO     Kedro project Brazilian Credit Card Spending                           ]8;id=928764;file:///usr/local/Caskroom/miniforge/base/envs/kedro-environment/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=88282;file:///usr/local/Caskroom/miniforge/base/envs/kedro-environment/lib/python3.10/site-packages/kedro/ipython/__init__.py#134\134]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=701734;file:///usr/local/Caskroom/miniforge/base/envs/kedro-environment/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=911005;file:///usr/local/Caskroom/miniforge/base/envs/kedro-environment/lib/python3.10/site-packages/kedro/ipython/__init__.py#135\135]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=853539;file:///usr/local/Caskroom/miniforge/base/envs/kedro-environment/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=462944;file:///usr/local/Caskroom/miniforge/base/envs/kedro-environment/lib/python3.10/site-packages/kedro/ipython/__init__.py#141\141]8;;\

In [73]:
expense_groups = catalog.load("params:expense_groups")
display(expense_groups)

# Create a function to assign labels based on expense groups
def assign_label(expense):
    for group, labels in expense_groups.items():
        if expense in labels:
            return group
    return "Other"

model_data['label'] = df['category_expense'].apply(assign_label)
model_data.label.value_counts()

[04/02/24 21:00:38] INFO     Loading data from params:expense_groups (MemoryDataset)...         ]8;id=120078;file:///usr/local/Caskroom/miniforge/base/envs/kedro-environment/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=936318;file:///usr/local/Caskroom/miniforge/base/envs/kedro-environment/lib/python3.10/site-packages/kedro/io/data_catalog.py#483\483]8;;\


{
    'Necessities': ['SERVI‚O', 'SUPERMERCADOS', 'FARMACIAS', 'POSTO DE GAS', 'M.O.T.O.'],
    'Discretionary': [
        'RESTAURANTE',
        'VAREJO',
        'VESTUARIO',
        'ARTIGOS ELETRO',
        'MOVEIS E DECOR',
        'AUTO PE‚AS',
        'JOALHERIA'
    ],
    'Travel': ['CIA AEREAS', 'HOTEIS', 'AGENCIA DE TUR']
}

                    WARNING  /var/folders/qn/pq9pzsn51vx7lzwtkc1r1hxr0000gn/T/ipykernel_2392/970702 ]8;id=246595;file:///usr/local/Caskroom/miniforge/base/envs/kedro-environment/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=902233;file:///usr/local/Caskroom/miniforge/base/envs/kedro-environment/lib/python3.10/warnings.py#109\109]8;;\
                             585.py:11: SettingWithCopyWarning:                                                    
                             A value is trying to be set on a copy of a slice from a DataFrame.                    
                             Try using .loc[row_indexer,col_indexer] = value instead                               
                                                                                                                   
                             See the caveats in the documentation:                                                 
                             https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#                
                             returning-a-view-versus-a-copy                                                        
                               model_data['label'] = df['category_expense'].apply(assign_label)                    
                                                                                                                   


label
Necessities      2835
Discretionary    1863
Other             130
Travel             72
Name: count, dtype: int64

# Visualization